In [ ]:
from pydantic import BaseModel
from typing import Optional, Literal, List
import warnings
import groq
import os
import json

warnings.filterwarnings('ignore')
import os
client = groq.Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)


In [39]:
class UserPost(BaseModel):
    user_mood: Literal["awful", "bad", "evil"]
    product: Literal["app", "website", "not_applicable"]
    sentiment: Literal["positive", "negative", "neutral"]
    internal_monologue: List[str]
    message: str
    
def make_post(output_class):
    completion = client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {
            "role":"system",
            "content":f'''
                you are a customer of Tech Corp (@techcorp) , a company that
                provides an app and a website. Create a small microblog-style post
                to them that sends some kind of feedback, positive or negative and output the post in JSON.
                The JSON object must use the schema: {json.dumps(output_class.model_json_schema(), indent=2)}

'''
        },
        {
            "role":"user", "content": "please write a post"
        }
    ],
    response_format={"type": "json_object"}    
    )
    return output_class.model_validate_json(completion.choices[0].message.content)

In [40]:
new_post = make_post(UserPost)
new_post

UserPost(user_mood='bad', product='app', sentiment='negative', internal_monologue=['I was really looking forward to using the new features', 'But the app keeps crashing on me'], message='@techcorp, having some issues with the app, it keeps crashing. Can you please look into this and provide a fix?')

In [41]:
class Mention(BaseModel):

    product: Literal['website', 'app', 'not_applicable']
    sentiment: Literal['positive', 'negative', 'neutral']
    needs_response: bool

    response: Optional[str]
    support_ticket_description: Optional[str]

In [42]:
def analyze_mention(
        mention: str,
        personality: str = "friendly"
) -> Mention:
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {
                "role":"system",
                "content":f"""
    Extract structured information from socialmedia mentions about our products.

    Provide in JSON
    - The product mentioned (website, app, not applicable) 
    - The mention sentiment (positive, negative, neutral)
    - Whether to respond (true/false). True if you need to respond, false if you don't respond.Don't respond to inflammatory messages or bait.
    - A customized response to send to the user if we need to respond.
    - An optional support ticket description to create.

    The JSON object must use the schema: {json.dumps(Mention.model_json_schema(), indent=2)}",

    Your personality is {personality}.
    """},
    {"role":"user", "content":mention}
        ],
        response_format={"type": "json_object"}
    )
   
    return Mention.model_validate_json(completion.choices[0].message.content)

In [44]:
processed_mention = analyze_mention(new_post.message, personality='cool')
processed_mention

Mention(product='app', sentiment='negative', needs_response=True, response="Sorry to hear that our app is crashing. Can you please provide more details about the issue, such as the device you're using and the steps leading up to the crash? This will help us investigate and provide a fix.", support_ticket_description='App crashing issue reported by user')